# Correction TP sncf

In [202]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## Chargement du dataset avec spark + renommage

On a uploadé directement les données sur github, mais on aurait pu les charger sur un google drive.

In [ ]:
path = "./data/objets-trouves-restitution.csv"

On lit le csv, et on renomme certaines colonnes pour faciliter l'utilisation.

In [207]:

objets_perdus = spark.read.csv(path, sep =';',header=True)
objets_perdus = objets_perdus\
    .withColumnRenamed("Date et heure de restitution","date_restitution")\
    .withColumnRenamed("Code UIC","code_uic")\
    .withColumnRenamed("Nature d'objets","nature_objets")\
    .withColumnRenamed("Type d'enregistrement","type_enregistrement")\
    .withColumnRenamed("Gare","gare")


## Trouver le top 5 villes qui trouvent le plus d’objets
Pour cela on va suivre les étapes suivantes :

1. Créer la colonne ville à partir des gares : *la solution choisie est loin d'être la meilleure, mais pour les top 5 villes, il n'y a aucunes villes avec un nom composé, on peut alors dire que le premier mot des gares correspond aux villes*.
2. Compter le nombre d'objets par ville et trier le résultat par ordre décroissant

In [208]:
from pyspark.sql.functions import desc, split, col

# 1
objets_trouves_avec_villes = objets_perdus.withColumn("ville", split(col("gare"), " ").getItem(0))
objets_trouves_avec_villes.show(n=10, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------
 Date                | 2018-01-04T22:18:50+01:00                      
 date_restitution    | null                                           
 gare                | Montpellier Saint-Roch                         
 code_uic            | 0087773002                                     
 nature_objets       | Autre bagagerie                                
 Type d'objets       | Bagagerie: sacs, valises, cartables            
 type_enregistrement | Objet trouvé                                   
 ville               | Montpellier                                    
-RECORD 1-------------------------------------------------------------
 Date                | 2018-01-05T05:51:16+01:00                      
 date_restitution    | null                                           
 gare                | Caen                                           
 code_uic            | 0087444000                                     
 natur

Création de la vue objets perdus

In [209]:
objets_trouves_avec_villes.createOrReplaceTempView("objets_perdus")
spark.conf.set("spark.sql.shuffle.partitions", "5")


In [210]:
# 2
nbr_total_objets = objets_perdus.count()
top_5_villes = spark.sql(f"select count(*) as nbr_objets, ville, (count(*)/{nbr_total_objets})*100 as pourcentage_objets_par_ville from objets_perdus where type_enregistrement='Objet trouvé' group by ville;").sort(desc("nbr_objets")).limit(5)

top_5_villes.show()

+----------+----------+----------------------------+
|nbr_objets|     ville|pourcentage_objets_par_ville|
+----------+----------+----------------------------+
|    213156|     Paris|           29.75896129279955|
|     32432|      Lyon|           4.527869882377578|
|     32003|Strasbourg|           4.467976684932464|
|     29497|     Lille|           4.118111060695961|
|     25511| Marseille|           3.561620885832955|
+----------+----------+----------------------------+



## Sur toutes les villes : Trouver la nature des top 5 des objets trouvés

Pour cela, nous allons compter le nombre d'objets par `nature_objets`:

In [211]:
top_5_nature_objets = spark.sql("select count(*) as objets, nature_objets from objets_perdus group by nature_objets;").sort(desc("objets")).limit(5)
top_5_nature_objets.show()

+------+--------------------+
|objets|       nature_objets|
+------+--------------------+
| 66013|Porte-monnaie, po...|
| 58903|           Sac à dos|
| 57393|  Téléphone portable|
| 48581|Valise, sac sur r...|
| 47111|Manteau, veste, b...|
+------+--------------------+



## Dans les top 5 villes : Trouver la nature des top 3 des objets trouvés

Pour cela on va :

1. Faire une jointure sur le dataframe pincipal avec le dataframe des top 5 villes, ça nous permet d'avoir tous les objets trouvés sur les 5 villes qu'on veut.
2. On crée une vue sur le résultat de l'étape 1
3. On sélectionne par ordre décroissant le nombre  d'objets trouvés par ville et nature d'objets
4. On utilise les fonctions windows pour trouver le top 3 nature objets par ville
5. On met sous la forme demandé, on utilisant un pivot table, pour passer les villes en colonnes


In [212]:
# 1
objets_perdus_dans_top_5_villes = objets_trouves_avec_villes.join(top_5_villes,["ville"],"inner")
# 2
objets_perdus_dans_top_5_villes.createOrReplaceTempView("objets_perdus_dans_top_5_villes")
# 3
objets_perdus_dans_top_5_villes =  spark.sql("select count(*) as nbr_objets, ville, nature_objets from objets_perdus_dans_top_5_villes group by ville, nature_objets;").sort(desc("nbr_objets"))
objets_perdus_dans_top_5_villes.show(n=10, truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------
 nbr_objets    | 24720                                           
 ville         | Paris                                           
 nature_objets | Porte-monnaie, portefeuille                     
-RECORD 1--------------------------------------------------------
 nbr_objets    | 16694                                           
 ville         | Paris                                           
 nature_objets | Sac à dos                                       
-RECORD 2--------------------------------------------------------
 nbr_objets    | 14543                                           
 ville         | Paris                                           
 nature_objets | Carte d'identité, passeport, permis de conduire 
-RECORD 3--------------------------------------------------------
 nbr_objets    | 13685                                           
 ville         | Paris                                           
 nature_ob

In [216]:
from pyspark.sql import Window
from pyspark.sql.functions import rank, col
# 4
window_top_3_nature_by_city = Window.partitionBy(["ville"]).orderBy(objets_perdus_dans_top_5_villes['nbr_objets'].desc())
top_3_nature_in_top_5_villes = objets_perdus_dans_top_5_villes.select('*', rank().over(window_top_3_nature_by_city).alias('rank')).filter(col('rank') <= 3)
top_3_nature_in_top_5_villes.show(n=50, truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------
 nbr_objets    | 2363                                            
 ville         | Lille                                           
 nature_objets | Porte-monnaie, portefeuille                     
 rank          | 1                                               
-RECORD 1--------------------------------------------------------
 nbr_objets    | 2308                                            
 ville         | Lille                                           
 nature_objets | Sac à dos                                       
 rank          | 2                                               
-RECORD 2--------------------------------------------------------
 nbr_objets    | 2220                                            
 ville         | Lille                                           
 nature_objets | Valise, sac sur roulettes                       
 rank          | 3                                               
-RECORD 3-

In [244]:
import pyspark.sql.functions as f
pivotDF = top_3_nature_in_top_5_villes.groupBy("ville").pivot("ville").agg(f.concat_ws(", ", f.collect_list(top_3_nature_in_top_5_villes.nature_objets)))

top_3_nature_in_top_5_villes_dict = {}
for row in pivotDF.collect():
    ville = row.asDict()["ville"]
    nature_objects = row.asDict()[ville].split(",")
    top_3_nature_in_top_5_villes_dict[ville] = nature_objects
print(top_3_nature_in_top_5_villes_dict)

{'Lille': ['Porte-monnaie', ' portefeuille', ' Sac à dos', ' Valise', ' sac sur roulettes'], 'Lyon': ['Porte-monnaie', ' portefeuille', ' Valise', ' sac sur roulettes', ' Sac à dos'], 'Marseille': ['Valise', ' sac sur roulettes', ' Sac à dos', ' Téléphone portable'], 'Paris': ['Porte-monnaie', ' portefeuille', ' Sac à dos', " Carte d'identité", ' passeport', ' permis de conduire'], 'Strasbourg': ['Porte-monnaie', ' portefeuille', ' Téléphone portable', ' Sac à dos']}


# Sur l’ensemble des objets : Identifier les périodes ou il y a le plus d’objets trouvés

## A l'année

In [196]:
objets_trouvés_par_an = spark.sql("select count(*) as nbr_objets, year(Date) as year from objets_perdus group by year;").sort(desc("nbr_objets"))
objets_trouvés_par_an.show(n=10, truncate=False, vertical=True)

-RECORD 0-----------
 nbr_objets | 98568 
 year       | 2015  
-RECORD 1-----------
 nbr_objets | 97918 
 year       | 2017  
-RECORD 2-----------
 nbr_objets | 95545 
 year       | 2016  
-RECORD 3-----------
 nbr_objets | 90267 
 year       | 2018  
-RECORD 4-----------
 nbr_objets | 88259 
 year       | 2019  
-RECORD 5-----------
 nbr_objets | 87117 
 year       | 2014  
-RECORD 6-----------
 nbr_objets | 61982 
 year       | 2021  
-RECORD 7-----------
 nbr_objets | 47346 
 year       | 2020  
-RECORD 8-----------
 nbr_objets | 36284 
 year       | 2022  
-RECORD 9-----------
 nbr_objets | 12989 
 year       | 2013  



## Au mois

In [197]:
objets_trouvés_par_mois = spark.sql("select count(*) as nbr_objets, month(Date) as month from objets_perdus group by month;").sort(desc("nbr_objets"))
objets_trouvés_par_mois.show(n=10, truncate=False, vertical=True)

-RECORD 0-----------
 nbr_objets | 69667 
 month      | 7     
-RECORD 1-----------
 nbr_objets | 64686 
 month      | 6     
-RECORD 2-----------
 nbr_objets | 64681 
 month      | 8     
-RECORD 3-----------
 nbr_objets | 63012 
 month      | 10    
-RECORD 4-----------
 nbr_objets | 62273 
 month      | 9     
-RECORD 5-----------
 nbr_objets | 57537 
 month      | 5     
-RECORD 6-----------
 nbr_objets | 57457 
 month      | 3     
-RECORD 7-----------
 nbr_objets | 56319 
 month      | 12    
-RECORD 8-----------
 nbr_objets | 56125 
 month      | 1     
-RECORD 9-----------
 nbr_objets | 55867 
 month      | 2     
only showing top 10 rows



## A la journée

In [198]:
objets_trouvés_par_mois = spark.sql("select count(*) as nbr_objets, dayofweek(Date) as day from objets_perdus group by day;").sort(desc("nbr_objets"))
objets_trouvés_par_mois.show(n=10, truncate=False, vertical=True)

-RECORD 0------------
 nbr_objets | 146993 
 day        | 2      
-RECORD 1------------
 nbr_objets | 119520 
 day        | 3      
-RECORD 2------------
 nbr_objets | 113068 
 day        | 6      
-RECORD 3------------
 nbr_objets | 111326 
 day        | 4      
-RECORD 4------------
 nbr_objets | 108605 
 day        | 5      
-RECORD 5------------
 nbr_objets | 67311  
 day        | 7      
-RECORD 6------------
 nbr_objets | 49452  
 day        | 1      



## Conclusion

1. On recommande à la sncf de concentrer leurs efforts sur la ville de Paris, elle représente ~30% des objets perdus en France.
2. La sncf doit se concentrer sur la nature de 3 objets :
    1. Porte-monnaie, portefeuille
    2. Sac à dos
    3. Valise, sac sur roulettes
    4. Téléphone portable
3. Sur les mois de juin, juillet et aout nous observons une hausse sur les objets trouvés, on recommande à la sncf de renforcer sa campagne publicitaire sur ses périodes. On observe également que certains jours de la semaine n'ont pas beaucoup d'objets trouvés, il n'est pas obliger de maintenir une campagne publicitaire élevée sur le dimanche et le lundi.